### Setup

In [28]:
exec(open("./variables.py").read())
exec(open(f"./util.py").read())

In [29]:
my_rtl = 'RCP'
# my_workloads = riscv_benchmarks_orig # + riscv_embench
my_w = 'towers' # stick to one workload for now
runparams = {
    'rtl': my_rtl,
    'bsub': 'bsub -q rhel7',
    'overwrite': True,
}

### Run RTL Sim

### Parse RTL Sim Waveforms

#### Header

In [30]:
exec(open(f"./util.py").read())
# use any of the workloads to print header info + dump idcodes, since that info is the same across all workloads
w = 'towers'
waveform_fpath = get_waveform_path(w,my_rtl)
header_fpath = get_outfile_path('headers',rtl=my_rtl)
print(f"./print_header {waveform_fpath} > {header_fpath}")


./print_header /tools/scratch/nayiri/power/chipyard-intech16-sep23/vlsi/output/chipyard.harness.TestHarness.RCP/towers.fsdb > /tools/C/nayiri/fsdb-parse/out/fsdb/headers/RCP.txt


#### Idcodes

In [31]:
dump_idcodes(rtl=my_rtl)

total signals: 21715
[num_signals] + [signal fsdb idcodes] --> /tools/C/nayiri/fsdb-parse/out/fsdb/idcodes/RCP.txt
[num_signals] + [(signal fsdb idcode, signal name(s)), ...] --> /tools/C/nayiri/fsdb-parse/out/fsdb/idcodes/signames-RCP.txt


#### Dump signal value for each workload

In [32]:
exec(open(f"./util.py").read())

idcode = 175523
waveform_fpath = get_waveform_path(my_w,my_rtl)
bin_file = get_outfile_path('sigvals',my_w,rtl=my_rtl, idcode=idcode)
print(f"""./dump_signal --fsdb {waveform_fpath} --idcode {idcode}  --out {bin_file}""")

./dump_signal --fsdb /tools/scratch/nayiri/power/chipyard-intech16-sep23/vlsi/output/chipyard.harness.TestHarness.RCP/towers.fsdb --idcode 175523  --out /tools/C/nayiri/fsdb-parse/out/fsdb/RCP/towers-idcode_175523.bin


### Load signal value

In [35]:
exec(open(f"./util.py").read())

times, sigvals = get_workload_sigvals(my_w,rtl=my_rtl,idcode=idcode)

In [36]:
# visually compare this output with spike log dump from RTL sim
for s in sigvals[:10]:
    print(bin(s))

0b10110111111001011000010100100011
0b11101110
0b10100010111
0b11111100000001010000010100010011
0b110000010101010001000001110011
0b110000000100000010001011110011
0b1000001001000101101001010010011
0b100101111001010010011
0b101000010001100011
0b110000001100000001000001110011
